# Ingest StackExchange data dumps

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LAION-AI/Open-Assistant/blob/main/notebooks/data-augmentation/stackexchange-builder/stackexchange-builder.ipynb)

This notebook takes a StackExchange Data dump "Posts.xml" file and ingests it into a Pandas Dataframe. Outputs of the file can be JSON, JSONL, Parquet, or CSV. 

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import json

# Extract StackExchange
Pull StackExchange file dumps. Specific column types are enforced to prevent errors on processing later in the notebook

In [ ]:
base_url = "https://ia600107.us.archive.org/view_archive.php?archive=/27/items/stackexchange/{0}&file=Posts.xml"


def get_all_filenames():
    response = requests.get("https://archive.org/download/stackexchange")
    if response.ok:
        soup = bs(response.content, "html.parser")
        table = soup.find("table")
        link_tags = table.find_all("a")
        urls = {}
        for link in link_tags:
            url = link["href"]
            name = url.split(".stackexchange")[0].replace(".", "_").replace("-", "_")
            if url.endswith("7z"):
                urls[name] = base_url.format(url)
        return urls


urls = get_all_filenames()

print(urls.keys())
print(urls.get("ai"))

dict_keys(['3dprinting_meta', '3dprinting', 'Stackoverflow_com_Posts_7z', 'academia_meta', 'academia', 'ai_meta', 'ai', 'android_meta', 'android', 'anime_meta', 'anime', 'apple_meta', 'apple', 'arduino_meta', 'arduino', 'askubuntu_com_7z', 'astronomy_meta', 'astronomy', 'aviation_meta', 'aviation', 'avp_meta', 'avp', 'beer_meta', 'beer', 'bicycles_meta', 'bicycles', 'bioacoustics_meta', 'bioacoustics', 'bioinformatics_meta', 'bioinformatics', 'biology_meta', 'biology', 'bitcoin_meta', 'bitcoin', 'blender_meta', 'blender', 'boardgames_meta', 'boardgames', 'bricks_meta', 'bricks', 'buddhism_meta', 'buddhism', 'cardano_meta', 'cardano', 'chemistry_meta', 'chemistry', 'chess_meta', 'chess', 'chinese_meta', 'chinese', 'christianity_meta', 'christianity', 'civicrm_meta', 'civicrm', 'codegolf_meta', 'codegolf', 'codereview_meta', 'codereview', 'coffee_meta', 'coffee', 'cogsci_meta', 'cogsci', 'computergraphics_meta', 'computergraphics', 'conlang_meta', 'conlang', 'cooking_meta', 'cooking', 'c

In [ ]:
xml_format_map = {
    "Id": int,
    "PostTypeId": int,
    "CreationDate": str,
    "Score": int,
    "ViewCount": int,
    "Body": str,
    "AnswerCount": int,
    "CommentCount": int,
    "ContentLicense": str,
    "AcceptedAnswerId": int,
    "ParentId": int,
}


# def extract_xml_file(file_url: str):
#   table = pd.read_xml(file_url)
#   return table


def xml_to_df(response: str):
    """
    Collect and Manually import XML into Dataframe

    pd.read_xml() errors when XML trees are too large, this is just a hack to
    download a XML file and parse into a Dataframe. **Not Tested on huge XML files**

    Parameters:
    response (Requests.Response): Requests response object with the XML data

    Returns:
    df (DataFrame): A Dataframe from the XML file
    """
    soup = bs(response.content, "xml")
    posts = soup.find_all("row")

    all_posts = [post.attrs for post in posts]

    df = pd.DataFrame(all_posts)
    df.AnswerCount.fillna(0, inplace=True)
    df.ViewCount.fillna(0, inplace=True)
    df.AcceptedAnswerId.fillna(0, inplace=True)
    df.ParentId.fillna(0, inplace=True)
    df["DataSource"] = response.url
    df = df.astype(xml_format_map)
    return df


dataset_name = "ai"

xml_posts_path = urls.get(dataset_name)


# df = extract_xml_file(test)
response = requests.get(xml_posts_path)
df = xml_to_df(response)


print(df.dtypes)
df.head()

Id                        int64
PostTypeId                int64
AcceptedAnswerId          int64
CreationDate             object
Score                     int64
ViewCount                 int64
Body                     object
OwnerUserId              object
LastEditorUserId         object
LastEditDate             object
LastActivityDate         object
Title                    object
Tags                     object
AnswerCount               int64
CommentCount              int64
ContentLicense           object
ParentId                  int64
ClosedDate               object
FavoriteCount            object
CommunityOwnedDate       object
LastEditorDisplayName    object
OwnerDisplayName         object
DataSource               object
dtype: object


,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,AnswerCount,CommentCount,ContentLicense,ParentId,ClosedDate,FavoriteCount,CommunityOwnedDate,LastEditorDisplayName,OwnerDisplayName,DataSource
0,1,1,3,2016-08-02T15:39:14.947,10,710,"<p>What does ""backprop"" mean? Is the ""backprop...",8,2444,2019-11-16T17:56:22.093,...,5,0,CC BY-SA 4.0,0,NaN,NaN,NaN,NaN,NaN,https://ia600107.us.archive.org/view_archive.p...
1,2,1,9,2016-08-02T15:40:20.623,14,1008,<p>Does increasing the noise in data help to i...,8,2444,2019-02-23T22:36:19.090,...,3,0,CC BY-SA 4.0,0,NaN,NaN,NaN,NaN,NaN,https://ia600107.us.archive.org/view_archive.p...
2,3,2,0,2016-08-02T15:40:24.820,15,0,"<p>""Backprop"" is the same as ""backpropagation""...",4,NaN,NaN,...,0,0,CC BY-SA 3.0,1,NaN,NaN,NaN,NaN,NaN,https://ia600107.us.archive.org/view_archive.p...
3,4,1,12,2016-08-02T15:41:22.020,33,1266,"<p>When you're writing your algorithm, how do ...",8,2444,2021-01-19T23:54:07.813,...,4,0,CC BY-SA 3.0,0,NaN,NaN,NaN,NaN,NaN,https://ia600107.us.archive.org/view_archive.p...
4,6,1,20,2016-08-02T15:43:35.460,7,279,<p>Given the following definition of an intell...,29,2444,2019-06-15T18:25:58.513,...,2,0,CC BY-SA 4.0,0,NaN,NaN,NaN,NaN,NaN,https://ia600107.us.archive.org/view_archive.p...


# Transformations

In [ ]:
def filter_only_questions_with_accepted_answers(df):
    """**TODO**
    Filter only to Questions with Accepted Answers

    Filter dataframe by questions that have accepted answers, should also include
    all rows of answers for those questions, even if not accepted.

    Parameters:
    df (DataFrame): containing a "AcceptedAnswerId", "Id", and "ParentId" columns

    Returns:
    df (DataFrame): current dataframe with filtered results
    """
    df = df[(df["AcceptedAnswerId"].notnull()) | (df["ParentId"] == df["Id"])]


def filter_scores_above(df, question_score_threshold: int = 20, answer_score_threshold: int = 20):
    """**TODO**
    Filter Dataframe by minimum scores

    Filter Question and Answer columns by score thresholds to trim lower scoring results

    Parameters:
    df (DataFrame): containing a "Score" column

    Returns:
    df (DataFrame): current dataframe with filtered results
    """
    df = df[
        ((df["Score"] >= question_score_threshold) & (df.PostTypeId == 1))
        | ((df["Score"] >= answer_score_threshold) & (df.PostTypeId == 2))
    ]


def convert_html_to_text(df, column: str = "Body"):
    """
    Convert HTML tags to pure text

    Feeds HTML text body into BeautifulSoup to parse it to only text. Set aside as
    function to provide option to skip

    Parameters:
    df (DataFrame): containing a "Body" column with HTML

    Returns:
    df (DataFrame): current dataframe with parsed column
    """
    df.dropna(subset=[column], inplace=True)
    df[f"{column}Clean"] = df[column].apply(lambda row: bs(row, "html.parser").text)


def clean_tags(df):
    """
    Convert Tags into Comma separated

    Converts Tag slugs into commas separated tags

    Parameters:
    df (DataFrame): containing a "Tags" column with slugs

    Returns:
    df (DataFrame): current dataframe with parsed column
    """
    df["TagsClean"] = df["Tags"].str.replace("-", " ").str.replace("><", ", ").str.replace("<", "").str.replace(">", "")


# filter_only_questions_with_accepted_answers(df)
# filter_scores_above(df)
convert_html_to_text(df)
clean_tags(df)

df[["Body", "BodyClean", "Tags", "TagsClean"]]
# print(df.shape)

,Body,BodyClean,Tags,TagsClean
0,"<p>What does ""backprop"" mean? Is the ""backprop...","What does ""backprop"" mean? Is the ""backprop"" t...",<neural-networks><backpropagation><terminology...,"neural networks, backpropagation, terminology,..."
1,<p>Does increasing the noise in data help to i...,Does increasing the noise in data help to impr...,<neural-networks><machine-learning><statistica...,"neural networks, machine learning, statistical..."
2,"<p>""Backprop"" is the same as ""backpropagation""...","""Backprop"" is the same as ""backpropagation"": i...",NaN,NaN
3,"<p>When you're writing your algorithm, how do ...","When you're writing your algorithm, how do you...",<neural-networks><hyperparameter-optimization>...,"neural networks, hyperparameter optimization, ..."
4,<p>Given the following definition of an intell...,Given the following definition of an intellige...,<philosophy><definitions><intelligent-agent>,"philosophy, definitions, intelligent agent"
...,...,...,...,...
23174,<p>The purpose of evaluating the state and act...,The purpose of evaluating the state and action...,NaN,NaN
23175,"<p>In machine translation, convolution is a te...","In machine translation, convolution is a techn...",NaN,NaN
23176,<p>One of the key features of ChatGPT is its a...,One of the key features of ChatGPT is its abil...,NaN,NaN
23177,<p>Given a neural network model for Covid-19 c...,Given a neural network model for Covid-19 clas...,<neural-networks><homework>,"neural networks, homework"


This groups questions with answers so that a row with a question also has a column with an answer. It then creates an AcceptedAnswerFlag column that is True if the answer was accepted by the person who asked the question. Changing the `number_of_results` variable will limit the number of answers you want to keep.

In [ ]:
questions = df[df.PostTypeId == 1]
answers = df[df.PostTypeId == 2]

df = pd.merge(
    questions,
    answers[
        [
            "Id",
            "CreationDate",
            "Score",
            "ViewCount",
            "CommentCount",
            "ContentLicense",
            "TagsClean",
            "BodyClean",
            "ParentId",
        ]
    ],
    left_on="Id",
    right_on="ParentId",
    suffixes=("_q", "_a"),
    how="left",
)

df["AcceptedAnswerFlag"] = df.apply(lambda row: row["Id_a"] == row["AcceptedAnswerId"], axis=1)

df = df.rename(
    columns={
        "BodyClean_q": "Question",
        "Score_q": "QuestionScore",
        "TagsClean_q": "QuestionTags",
        "BodyClean_a": "Answer",
        "Score_a": "AnswerScore",
        "ContentLicense_q": "QuestionContentLicense",
        "ContentLicense_a": "AnswerContentLicense",
        "CreationDate_q": "CreationDate",
    }
)

## Set the number of results to a lower number to only return top N rated Answers.
number_of_results = 25
df = (
    df.sort_values(by=["AcceptedAnswerFlag", "AnswerScore"], ascending=[False, False])
    .groupby("Question")
    .head(number_of_results)
    .reset_index(drop=True)
)

df[["Id_q", "Question", "QuestionScore", "QuestionTags", "Id_a", "Answer", "AnswerScore", "AcceptedAnswerFlag"]].head()

,Id_q,Question,QuestionScore,QuestionTags,Id_a,Answer,AnswerScore,AcceptedAnswerFlag
0,1768,"In Portal 2 we see that AI's can be ""killed"" b...",175,"philosophy, decision theory, mythology of ai, ...",1769.0,This classic problem exhibits a basic misunder...,146.0,True
1,10623,What is self-supervised learning in machine le...,91,"machine learning, comparison, supervised learn...",10624.0,Introduction\nThe term self-supervised learnin...,90.0,True
2,111,"Obviously, self-driving cars aren't perfect, s...",100,"philosophy, ethics, autonomous vehicles, decis...",1790.0,\nHow could self-driving cars make ethical dec...,76.0,True
3,14224,If the original purpose for developing AI was ...,69,"philosophy, social, explainable ai",14247.0,"As argued by Selvaraju et al., there are three...",75.0,True
4,1479,Do scientists or research experts know from th...,94,"neural networks, deep learning, convolutional ...",4044.0,There are many approaches that aim to make a t...,69.0,True


In [ ]:
testing_id = df.Id_q.mode()[0]
df[(df.Id_q == testing_id) | (df.ParentId_a == testing_id)][
    ["Id_q", "Question", "ParentId_a", "AcceptedAnswerId", "Id_a", "Answer", "AnswerScore", "AcceptedAnswerFlag"]
]
# df[['Id_q', 'Question', 'ParentId_a', 'AcceptedAnswerId', 'Id_a', 'Answer', 'AnswerScore', 'AcceptedAnswerFlag']]

,Id_q,Question,ParentId_a,AcceptedAnswerId,Id_a,Answer,AnswerScore,AcceptedAnswerFlag
7,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15744.0,I think this is a fairly common misconception ...,62.0,True
3662,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15753.0,I think your premise is flawed.\nYou seem to a...,19.0,False
3713,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15747.0,TL;DR: The subtleties of infinity are made app...,12.0,False
3788,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15756.0,"In Haskell, you can type:\nprint [1..]\nand it...",9.0,False
3821,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15758.0,I believe humans can be said to understand inf...,8.0,False
3882,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15762.0,(There's a summary at the bottom for those who...,7.0,False
4389,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15783.0,"Then premise assumes that humans ""understand"" ...",4.0,False
4849,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15740.0,By adding some rules for infinity in arithmeti...,3.0,False
4850,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15803.0,I think the concept that is missing in the dis...,3.0,False
5763,15730,"As a human being, we can think infinity. In pr...",15730.0,15744,15768.0,"Computers don't understand ""infinity"" or even ...",2.0,False


# Create JSONL version of Dataframe
This groups the dataframe by question data and creates nested list of Answers for that group. The entire list contains individual JSON objects, each representing a single question in the dataset with a key, Answers, which contains a list of dictionaries for each answer to the question.

In [ ]:
j = (
    df.groupby(
        ["Title", "Question", "QuestionScore", "QuestionTags", "QuestionContentLicense", "DataSource", "CreationDate"]
    )
    .apply(lambda x: x[["Answer", "AnswerScore", "AcceptedAnswerFlag"]].to_dict("records"))
    .reset_index()
    .rename(columns={0: "Answers"})
    .to_json(orient="records")
)

data = json.loads(j)

for post in data:
    if len(post.get("Answers")) >= 4:
        print(json.dumps(post, indent=4))
        break

{
    "Title": "1 hidden layer with 1000 neurons vs. 10 hidden layers with 100 neurons",
    "Question": "These types of questions may be problem-dependent, but I have tried to find research that addresses the question whether the number of hidden layers and their size (number of neurons in each layer) really matter or not.\nSo my question is, does it really matter if we for example have 1 large hidden layer of 1000 neurons vs. 10 hidden layers with 100 neurons each?\n",
    "QuestionScore": 16,
    "QuestionTags": "neural networks",
    "QuestionContentLicense": "CC BY-SA 3.0",
    "DataSource": "https://ia600107.us.archive.org/view_archive.php?archive=/27/items/stackexchange/ai.stackexchange.com.7z&file=Posts.xml",
    "CreationDate": "2017-05-04T13:06:37.990",
    "Answers": [
        {
            "Answer": "Basically, having multiple layers (aka a deep network) makes your network more eager to recognize certain aspects of input data. For example, if you have the details of a house

# Save file

Files can be saved as JSON, JSONL, CSV, or Parquet

In [ ]:
file_name = dataset_name


def save_data(data: list, file_name: str, file_type: str = ["csv", "json", "jsonl", "parquet"]):
    """
    Save Data to file

    Save Data list to file as either JSON or JSONL

    Parameters:
    data (list): list of dictionaries
    file_name (str): name of file (no extension)
    jsonl (bool): to save file as either JSON or JSONL
    """
    file_type = file_type.lower()

    if file_type == "csv" and isinstance(data, pd.DataFrame):
        data.to_csv(f"/content/{file_name}.csv", index=False)

    elif file_type == "json" and isinstance(data, list):
        print(json.dumps(data, indent=4), file=open(f"/content/{file_name}.json", "w"))

    elif file_type == "jsonl" and isinstance(data, list):
        for item in data:
            print(json.dumps(item), file=open(f"/content/{file_name}.jsonl", "a"))

    elif file_type == "parquet" and isinstance(data, pd.DataFrame):
        data.to_parquet(f"/content/{file_name}.parquet", index=False)

    else:
        print("Data should be either of List type for JSON and JSONL, or Pandas Dataframes for CSV and Parquet")


# save_data(data=data, file_name=file_name, file_type='jsonl')
# save_data(data=df, file_name=file_name, file_type='parquet')

Data should be either of List type for JSON and JSONL, or Pandas Dataframes for CSV and Parquet


# Open-Assistant Data Scheme

Testing putting the data into the Open-Assistant Data Scheme

https://github.com/LAION-AI/Open-Assistant/blob/main/docs/data_schemas.md

In [ ]:
from typing import TypeVar, List, Dict, Any, Literal
from json import JSONEncoder

T = TypeVar("T", bound="ConversationTreeNode")


class ConversationTreeNode:
    text: str  # The text of the node
    role: Literal["prompter", "assistant"]  # Whether the node is a user prompt/follow-up or an assistant response
    children: List[T]  # The children of the node (if you have a linear conversation, this will be of length 0 or 1)
    metadata: Dict[str, Any]  # Node metadata (see below)

    def __init__(
        self, text: str, role: Literal["prompter", "assistant"], children: List[T], metadata: Dict[str, Any]
    ) -> None:
        self.text = text
        self.role = role
        self.children = children
        self.metadata = metadata


class ConversationTree:
    root: ConversationTreeNode  # The node containing the initial prompt
    metadata: Dict[str, Any]  # Tree metadata, different from root node metadata.

    def __init__(self, root: ConversationTreeNode, metadata: Dict[str, Any]) -> None:
        self.root = root
        self.metadata = metadata


# subclass JSONEncoder
class TreeEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__

In [ ]:
conversation_forest = []

tree_metadata_map = {"Title": str, "QuestionContentLicense": str, "DataSource": str, "CreationDate": str}
question_metadata_map = {"QuestionScore": int, "QuestionTags": str}
answer_metadata_map = {"AnswerScore": int, "AcceptedAnswerFlag": bool}


for item in data:
    prompt = item.get("Question")
    metadata = {k: v for k, v in item.items() if k in question_metadata_map}
    root = ConversationTreeNode(text=prompt, role="prompter", children=[], metadata=metadata)

    for answer in item.get("Answers"):
        response = answer.get("Answer")
        metadata = {k: v for k, v in answer.items() if k in answer_metadata_map}
        child = ConversationTreeNode(text=response, role="assistant", children=[], metadata=metadata)
        root.children.append(child)

    metadata = {k: v for k, v in item.items() if k in tree_metadata_map}
    conversation_tree = ConversationTree(root=root, metadata=metadata)
    conversation_forest.append(conversation_tree)


conversation_forest_json = [
    json.loads(TreeEncoder().encode(conversation_tree)) for conversation_tree in conversation_forest
]


# print(json.dumps(conversation_forest_json[0], indent=4))


print(json.dumps(conversation_forest_json, indent=4), file=open(f"/content/{file_name}.json", "w"))

{
    "root": {
        "text": "Science Fiction has frequently shown AI to be a threat to the very existence of mankind. AI systems have often been the antagonists in many works of fiction, from 2001: A Space Odyssey through to The Terminator and beyond.\nThe Media seems to buy into this trope as well.  And in recent years we have had people like Elon Musk warn us of the dangers of an impending AI revolution, stating that AI is more dangerous than nukes.\nAnd, apparently, experts think that we will be seeing this AI revolution in the next 100 years.\nHowever, from my (albeit limited) study of AI, I get the impression that they are all wrong. I am going to outline my understanding below, please correct me if I am wrong:\n\nFirstly, all of these things seem to be confusing Artificial Intelligence with Artificial Consciousness.  AI is essentially a system to make intelligent decisions, whereas AC is more like the \"self-aware\" systems that are shown in science fiction.\n\nNot AI itself,